In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  41.9M      0  0:00:01  0:00:01 --:--:-- 41.9M


In [2]:
!rm -r aclImdb/train/unsup

In [3]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [4]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  # Shuffle the list of training files using a seed, to ensure we get the same validation set every time we run the code.
  random.Random(1337).shuffle(files)
  # take 20% files of the training files to use for validation.
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  # move the files to aclImdb/val/neg and aclImdb/val/pos
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [5]:
!cat aclImdb/val/pos/10010_7.txt

cat: aclImdb/val/pos/10010_7.txt: No such file or directory


In [6]:
from tensorflow import keras
batch_size = 32

# Each subfolder name becomes a class label.
# Keras automatically sorts folder names alphabetically to determine numeric labels

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size = batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size = batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size = batch_size
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [7]:
text_only_train_ds = train_ds.map(lambda x, y:x)

from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    # Truncate long sequences to 600 tokens,
    # Pad short sequences to 600 tokens
    # orders by frequency and assign integer
    max_tokens = max_tokens,
    output_mode = "int",
    output_sequence_length = max_length,
)

text_vectorization.adapt(text_only_train_ds)
# Integer encoding
int_train_ds = train_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)


int_val_ds = val_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)


int_test_ds = test_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)

In [8]:
for inputs, targets in int_train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", inputs.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 600)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int64'>
inputs[0]: tf.Tensor(
[   11    18     7  1210    10   284     9     2    81   262    16    55
  3258     1    27   299     9    14  2697    19    10   103     9     1
    10   389    30    38   655  2939  5106 18063     7    38  2438     3
  1465    27    92     4    18   480   979   144   385   352    95    39
     2  2513     3   615    42    55  1638    76  1294     9    97    26
    75   124   147    10    39  7986     3  6755    10    78    39    12
   192     1    12     2   162   134  2869    10   103   233     2  7891
   224   235    12  5798  8326 17438   224    22    61   121    11    18
   164    62   655    39   144     1    93    55  1262    92     5    55
  3264    52    10   666  1383    19     9   668     4  1289   785    16
  4109     5   102    12   287     6    28   480  5829    19    35    32
   880    58     8     2   129    30  1288    86    11  3823     7 

In [9]:
inputs= keras.Input(shape = (None,), dtype = "int64")

# word index -> embedding layer -> each word is a 256-dimensional vector # (20000, 256)
embedded = layers.Embedding(input_dim = max_tokens, output_dim = 256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer = "rmsprop",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only = True)
]

model.fit(int_train_ds,
          validation_data = int_val_ds,
          epochs = 10,
          callbacks = callbacks)

model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")
# 87.1%

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 36s 49ms/step - accuracy: 0.6413 - loss: 0.6197 - val_accuracy: 0.8220 - val_loss: 0.4335
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.8288 - loss: 0.4216 - val_accuracy: 0.8520 - val_loss: 0.3762
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 49ms/step - accuracy: 0.8699 - loss: 0.3360 - val_accuracy: 0.8812 - val_loss: 0.3058
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 53ms/step - accuracy: 0.8913 - loss: 0.2974 - val_accuracy: 0.8512 - val_loss: 0.3500
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.9100 - loss: 0.2546 - val_accuracy: 0.8888 - val_loss: 0.2964
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.9217 - loss: 0.2233 - val_accuracy: 0.8864 - val_loss: 0.3297
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.9346 - loss: 0.1937 - val_accuracy: 0.8906 - val_loss: 0.3170
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 48ms/step - accuracy: 0.9450 - loss: 0.1617 - 

In [9]:
embedding_layer = layers.Embedding(input_dim = 10, output_dim = 256, mask_zero = True)
some_input = [
    [4,5,6,7,0,0,0],
    [5,6,1,0,0,0, 0],
    [4,5,0,0,0,0,0]
]

mask = embedding_layer.compute_mask(some_input)
# the timestep will be skipped if mask[i,t] is 0 or False, and proceed otherwise

In [10]:
# Using an Embedding layer with masking enabled

# [10, 92, 104, 7, 0, 0, 0, 0]
inputs = keras.Input(shape = (None, ), dtype = "int")

# [v10, v92, v104, v7, v0, v0, v0]
embedded = layers.Embedding(input_dim = max_tokens, output_dim = 256, mask_zero = True)(inputs)
# [1, 1, 1, 1, 0, 0, 0]
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 256) │  5,120,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 64)        │     73,984 │ embedding_1[0][0… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
5120000/256

20000.0

In [12]:
73984 / 64

1156.0

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only = True)
]

model.fit(int_train_ds,
          validation_data = int_val_ds,
          epochs = 10,
          callbacks = callbacks)

model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")
# 87.7%

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.6841 - loss: 0.5703 - val_accuracy: 0.8472 - val_loss: 0.3607
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8568 - loss: 0.3426 - val_accuracy: 0.8816 - val_loss: 0.2867
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 48ms/step - accuracy: 0.8903 - loss: 0.2726 - val_accuracy: 0.8614 - val_loss: 0.3374
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.9140 - loss: 0.2266 - val_accuracy: 0.8872 - val_loss: 0.2782
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.9330 - loss: 0.1791 - val_accuracy: 0.8866 - val_loss: 0.3028
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 48ms/step - accuracy: 0.9501 - loss: 0.1361 - val_accuracy: 0.8682 - val_loss: 0.3320
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 49ms/step - accuracy: 0.9622 - loss: 0.1081 - val_accuracy: 0.8834 - val_loss: 0.3573
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.9744 - loss: 0.0769 - 

In [14]:
# USING PRETRAINED WORD EMBEDDINGS
# when training data is low, use pretrained word embeddings that are highly structured and exhibits useful properties - captures genetric aspects of language structure

# Global Vectors for Word Representation
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


--2025-11-29 02:22:13--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-11-29 02:22:13--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.00MB/s    in 2m 39s  

2025-11-29 02:24:52 (5.18 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [15]:
# Parsing the GloVe word-embeddings file

import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)

    # String containing many numbers separated by spaces
    # "0.12 -0.45 0.88 0.03 ... "
    # Converts them into a NumPy array
    coefs = np.fromstring(coefs, "f", sep = " ")
    embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors")

Found 400000 word vectors


In [19]:
# Preparing the GloVe

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary)))) # use it to create a mapping from words to their index in the vocabulary

embedding_matrix = np.zeros((max_tokens, embedding_dim)) # prepare a matrix that we'll fill with the GloVe vectors.
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)

  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    # Fill entry i in the matrix with the word vector for index i.
    # Words not found in the embedding index will be all zeros.

embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer = keras.initializers.Constant(embedding_matrix),
    trainable = False,
    mask_zero = True
)

In [20]:
# Model that uses a pretrained Embedding layer
inputs = keras.Input(shape = (None,), dtype = "int64")

embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 100) │  2,000,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None)      │          0 │ input_layer_2[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 64)        │     34,048 │ embedding_3[0][0… │
│ (Bidirectional)     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,034,113 (7.76 MB)

 Trainable params: 34,113 (133.25 KB)

 Non-trainable params: 2,000,000 (7.63 MB)

In [21]:
callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only = True)
]

model.fit(int_train_ds,
          validation_data = int_val_ds,
          epochs = 10,
          callbacks = callbacks)

model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")
# 87.2 %

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.6224 - loss: 0.6420 - val_accuracy: 0.7956 - val_loss: 0.4563
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.7722 - loss: 0.4868 - val_accuracy: 0.8176 - val_loss: 0.4069
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 52ms/step - accuracy: 0.8094 - loss: 0.4289 - val_accuracy: 0.8460 - val_loss: 0.3542
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.8357 - loss: 0.3873 - val_accuracy: 0.7760 - val_loss: 0.4527
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 48ms/step - accuracy: 0.8463 - loss: 0.3575 - val_accuracy: 0.8692 - val_loss: 0.3153
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.8568 - loss: 0.3368 - val_accuracy: 0.8498 - val_loss: 0.3447
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.8688 - loss: 0.3190 - val_accuracy: 0.8764 - val_loss: 0.3001
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.8779 - loss: 0.3040 - 